## Import libraries

In [1]:
import sys
import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()
from google.colab import drive
import shutil

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm
import pickle

from __future__ import print_function
import argparse
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


# Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/ROB535_group_project/GTSRB

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GTSRB  Meta.csv       Test	test_data.pkl  Train.csv
Meta   meta_data.pkl  Test.csv	Train	       train_data.pkl


In [5]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "ROB535_group_project/GTSRB"
GOOGLE_DRIVE_PATH = '/content/drive/My Drive/ROB535_group_project/GTSRB'
#shared_folder_path = '/content/drive/My Drive/ROB535_group_project/GTSRB'
print(os.listdir(GOOGLE_DRIVE_PATH))

['Meta.csv', 'Test.csv', 'Train.csv', 'Meta', 'Test', 'Train', 'train_data.pkl', 'test_data.pkl', 'meta_data.pkl', 'GTSRB']


In [6]:
# To save time, just take 10% of train and test. In final run, change this to 1.0
fraction = 0.1

In [7]:
import cv2
import numpy as np
from tqdm import tqdm

def img_preprocess(df, path):
    X = []
    total_images = len(df['Path'])

    with tqdm(total=total_images, desc="Loading images", unit="image") as pbar:
        for img_path in df['Path']:
            img = cv2.imread(path + '/' + str(img_path))
            if img is None:
                print(f"Failed to load image: {img_path}")
                pbar.update(1)
                continue
            img = cv2.resize(img, (30, 30))
            # img = img / 255.0
            X.append(img)
            pbar.update(1)

    X = np.array(X)
    return X


In [8]:
%cd ../
!rm -rf /content/gtsrb-pytorch
!cp -r /content/drive/My\ Drive/gtsrb-pytorch /content/
%cd /content/gtsrb-pytorch
%ls

/
/content/drive/.shortcut-targets-by-id/13OmO7R4bavH7GvDK5zwGHdUl_WXMb1GR/gtsrb-pytorch
data.py  evaluate.py  main.py  model/  model.py  __pycache__/  README.md


In [9]:
data_path = GOOGLE_DRIVE_PATH
!ln -s "$data_path" data
print(os.listdir(data_path))

['Meta.csv', 'Test.csv', 'Train.csv', 'Meta', 'Test', 'Train', 'train_data.pkl', 'test_data.pkl', 'meta_data.pkl', 'GTSRB']


In [10]:
#!python main.py --data data --epochs 40

In [11]:
!python evaluate.py --data data --model model/model_40.pth
%ls

  0% 0/1263 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100% 1263/1263 [08:06<00:00,  2.60it/s]
Accuracy: 99.84%
Succesfully wrote pred.csv, you can upload this file to the kaggle competition at https://www.kaggle.com/c/nyu-cv-fall-2017/
data@  data.py  evaluate.py  main.py  model/  model.py  pred.csv  __pycache__/  README.md


In [12]:
!cat pred.csv

Filename,ClassId
00020,4,4
06259,4,4
00528,18,18
11686,17,17
07957,8,8
04208,31,31
04960,7,7
07771,8,8
08762,9,9
09590,7,7
11118,13,13
10652,1,1
12240,35,35
09616,35,35
10387,5,5
02287,17,17
04910,13,13
01752,31,31
11690,25,25
10480,18,18
00367,38,38
00067,2,2
07559,30,30
05619,3,3
02916,1,1
07689,12,12
08429,10,10
00715,6,6
03467,38,38
12137,5,5
00512,25,25
07658,17,17
07177,17,17
07881,10,10
06282,7,7
09259,35,35
02919,1,1
04770,38,38
09886,4,4
02878,13,13
08767,4,4
07772,13,13
02794,34,34
09539,3,3
06023,4,4
00526,3,3
03478,11,11
08188,38,38
10142,28,28
02487,40,40
10268,4,4
03160,29,29
08322,29,29
05348,4,4
11182,11,11
08091,2,2
10543,13,13
03752,28,28
04039,5,5
08338,10,10
07669,2,2
00968,7,7
04072,8,8
11478,4,4
07161,2,2
02227,8,8
02396,24,24
01361,3,3
06254,1,1
10678,13,13
10323,7,7
05250,38,38
00472,3,3
08094,5,5
10903,17,17
01967,4,4
03158,10,10
07707,4,4
10735,42,42
10357,3,3
07865,10,10
01470,15,15
05564,8,8
08372,33,33
07782,8,8
11333,13,13
05862,12,12
02209,23,23
07774,3,3

In [13]:
%ls
!cp pred.csv /content/drive/My\ Drive/ROB535_group_project

data@  data.py  evaluate.py  main.py  model/  model.py  pred.csv  __pycache__/  README.md
